# GAB model: groundwater + heat flow

Each layer corresponds to the region between two surfaces as defined in `UW-GAB_layer_parameters.csv`. Each layer is assigned a "material index" which is used to map the thermal and hydraulic properties:

- $k_h$: hydraulic conductivity (m/s)
- $\phi$: porosity
- $k_T$: thermal conductivity (W/m/K)
- $H$: rate of heat production (W/m$^3$)

In [1]:
import numpy as np
import underworld as uw
import pandas as pd
import os
from scipy.interpolate import RegularGridInterpolator
import underworld.visualisation as vis

%matplotlib inline

In [2]:
### convert between units
from pint import UnitRegistry
u = UnitRegistry()

## Import datasets

In [3]:
### directory of stored data
data_dir = '/Users/temp/Documents/UW/GAB-Notebooks/data/'

numpy_directory = data_dir + "GAB_surfaces/NumPy/"
# png_directory = "../data/GAB_surfaces/png/"
surface_filename = numpy_directory + "{:s}.npz"

df_layers = pd.read_csv(data_dir + "GAB_surfaces/UW-GAB_layer_parameters.csv", index_col=0)


## Set up model dimensions and parameters

In [133]:
Tmin = 298.0
Tmax = 500.0
Nx, Ny, Nz = 20,20,50 # global size

# define bounding box
xmin, xmax, ymin, ymax = -955637.8812, 1034362.2443650428, 6342298.2975, 8922298.39436168
zmin, zmax = -8000.0, 1200.0

# resolution
dx, dy, dz = 20e3, 20e3, 50.
Nx, Ny, Nz = int((xmax-xmin)/dx), int((ymax-ymin)/dy), int((zmax-zmin)/dz)

if uw.mpi.rank == 0:
    print("global number of elements in x,y,z {} | total number of elements = {}".format((Nx,Ny,Nz), Nx*Ny*Nz))

global number of elements in x,y,z (99, 129, 18) | total number of elements = 229878


In [6]:
### file directory to store data

simulation_directory = "../simulations/"

if not os.path.exists(simulation_directory):
    os.makedirs(simulation_directory)

## Set up the mesh

Initialise a Q1 finite element mesh and mesh variables.

In [7]:
deformedmesh = True
elementType = "Q1"
mesh = uw.mesh.FeMesh_Cartesian( elementType = (elementType), 
                                 elementRes  = (Nx,Ny,Nz), 
                                 minCoord    = (xmin,ymin,zmin), 
                                 maxCoord    = (xmax,ymax,zmax)) 

gwHydraulicHead            = mesh.add_variable( nodeDofCount=1 )
temperatureField           = mesh.add_variable( nodeDofCount=1 )
temperatureField0          = mesh.add_variable( nodeDofCount=1 )
velocityField              = mesh.add_variable( nodeDofCount=3 )
heatProductionField        = mesh.add_variable( nodeDofCount=1 )



coords = mesh.data

Xcoords = np.unique(coords[:,0])
Ycoords = np.unique(coords[:,1])
Zcoords = np.unique(coords[:,2])
nx, ny, nz = Xcoords.size, Ycoords.size, Zcoords.size

In [48]:
meshMaterial               = mesh.add_variable( nodeDofCount=1 )

## Deform bottom of mesh to top of basement

We want to deform the $z$-axis spacing so that the bottom of the mesh follows the shape of the top of the basement.

In [8]:
# with np.load(surface_filename.format('W910_BASEMENT_v1')) as npz:
#     basement_interp = RegularGridInterpolator((npz['y'], npz['x']), np.flipud(npz['data']))
#     max_basement_coord = npz['data'].max()
    

    
    
# basement_shape = basement_interp((mesh.data[:,1], mesh.data[:,0]))

# # depth below which to deform
# z_deform = zmin


# with mesh.deform_mesh():
#     zcube = coords[:,2].reshape(nz,ny,nx)
#     zcube_norm = zcube.copy()
#     zcube_norm -= z_deform
#     zcube_norm /= zmax - z_deform
#     zcube_mask = zcube_norm < 0
    
#     # difference to add to existing z coordinates
#     dzcube = zcube_norm * -(zmax - basement_shape.reshape(zcube.shape))
    
#     mesh.data[:,2] += dzcube.ravel()
#     coords = mesh.data

In [9]:
# with np.load(surface_filename.format(df_layers['Layer name'].iloc[-2])) as npz:
#     basement_interp = RegularGridInterpolator((npz['y'], npz['x']), np.flipud(npz['data']))
    
# basement_shape = basement_interp((mesh.data[:,1], mesh.data[:,0]))

# ### deform mesh that is below the basement
# with mesh.deform_mesh():
#     ### add values 
#     mesh.data[:,2][mesh.data[:,2] < basement_shape] += basement_shape[mesh.data[:,2] < basement_shape] - mesh.data[:,2][mesh.data[:,2] < basement_shape]


## Deform mesh to surface topography

We want to deform the $z$-axis spacing so that the surface of the mesh is draped over the topography.

In [10]:
with np.load(surface_filename.format(df_layers['Layer name'].iloc[0])) as npz:
    topo_interp = RegularGridInterpolator((npz['y'], npz['x']), np.flipud(npz['data']))
    
    
local_topography = topo_interp((mesh.data[:,1], mesh.data[:,0]))

# depth above which to deform
z_deform = zmin

with mesh.deform_mesh():
    zcube = coords[:,2].reshape(nz,ny,nx)
    zcube_norm = zcube.copy()
    zcube_norm -= z_deform
    zcube_norm /= zmax - z_deform
    zcube_mask = zcube_norm < 0
    
    # difference to add to existing z coordinates
    dzcube = zcube_norm * -(zmax - local_topography.reshape(zcube.shape))
    
    mesh.data[:,2] += dzcube.ravel()
    coords = mesh.data

## Set up the types of boundary conditions

Set the left, right and bottom walls such that flow cannot pass through them, only parallel.
In other words for groundwater head $h$:

$ \frac{\partial h}{\partial x}=0$ : left and right walls

$ \frac{\partial h}{\partial y}=0$ : bottom wall

This is only solvable if there is topography or a non-uniform upper hydraulic head boundary condition.

In [11]:
topWall = mesh.specialSets["MaxK_VertexSet"]
bottomWall = mesh.specialSets["MinK_VertexSet"]

gwPressureBC = uw.conditions.DirichletCondition( variable      = gwHydraulicHead, 
                                               indexSetsPerDof = ( topWall   ) )

temperatureBC = uw.conditions.DirichletCondition( variable        = temperatureField,
                                                  indexSetsPerDof = (topWall+bottomWall))

In [12]:
# create a linear gradient [0, 1] top to bottom of mesh
znorm = mesh.data[:,2].copy()
znorm -= zmin
znorm /= (zmax-zmin)
linear_gradient = 1.0 - znorm

# pressure and temperature initial conditions
initial_pressure = linear_gradient*(zmax-zmin)
initial_temperature = linear_gradient*(Tmax - Tmin) + Tmin
initial_temperature = np.clip(initial_temperature, Tmin, Tmax)

gwHydraulicHead.data[:]  = initial_pressure.reshape(-1,1)
temperatureField.data[:] = initial_temperature.reshape(-1,1)

# assign BCs (account for pressure of water below sea level)
sealevel = 0.0
seafloor = topWall[mesh.data[topWall,2] < sealevel]

gwHydraulicHead.data[topWall] = 0.
gwHydraulicHead.data[seafloor] = -((mesh.data[seafloor,2]-sealevel)*1.0).reshape(-1,1)


temperatureField.data[topWall] = Tmin
temperatureField.data[bottomWall] = Tmax

### Import water table surface

In [13]:
with np.load(data_dir + '/water_table_surface.npz') as npz:
    wt = npz['data']
    wt_x = npz['x']
    wt_y = npz['y']

rgi_wt = RegularGridInterpolator((wt_y, wt_x), wt)

wt_interp = rgi_wt(mesh.data[topWall,0:2][:,::-1])

In [14]:
zCoordFn = uw.function.input()[2]

gwHydraulicHead.data[:] = zCoordFn.evaluate(mesh)

gwHydraulicHead.data[topWall] += (-wt_interp).reshape(-1,1)




## Set up particle swarm

Each cell contains particles that _must_ be assigned isotropic thermal and hydraulic properties.

> __Four__ particles per cell seems to prevent the model from crashing.

In [15]:
gaussPointCount = 1
gaussPointPerCell = gaussPointCount**mesh.dim

swarm         = uw.swarm.Swarm( mesh=mesh )
swarmLayout   = uw.swarm.layouts.PerCellGaussLayout(swarm=swarm, gaussPointCount=gaussPointCount)
swarm.populate_using_layout( layout=swarmLayout )

In [16]:
materialIndex  = swarm.add_variable( dataType="int",    count=1 )
cellCentroid   = swarm.add_variable( dataType="double", count=3 )
swarmVelocity  = swarm.add_variable( dataType="double", count=3 )

hydraulicDiffusivity    = swarm.add_variable( dataType="double", count=1 )
fn_hydraulicDiffusivity = swarm.add_variable( dataType="double", count=1 )
thermalDiffusivity      = swarm.add_variable( dataType="double", count=1 )
heatProduction          = swarm.add_variable( dataType="double", count=1 )
a_exponent              = swarm.add_variable( dataType="double", count=1 )

### used on the velocity field to caculate darcy flow
porosity                = mesh.add_variable( nodeDofCount=1 )


In [17]:
# find the centroids using a single gaussPointCount per cell (parallel-safe, maybe)

swarm0         = uw.swarm.Swarm( mesh=mesh )
swarmLayout0   = uw.swarm.layouts.PerCellGaussLayout(swarm=swarm0, gaussPointCount=1)
swarm0.populate_using_layout( layout=swarmLayout0 )

cell_centroids = swarm0.data[:]
cellCentroid.data[:] = np.repeat(cell_centroids, gaussPointPerCell, axis=0)

## Import geological surfaces

Assign a material index for all cell centroids which lie between two surfaces

## Assign material properties

Use level sets to assign hydraulic diffusivities to a region on the mesh corresponding to any given material index.

- $H$       : rate of heat production
- $\rho$     : density
- $k_h$     : hydraulic conductivity
- $k_t$     : thermal conductivity
- $\kappa_h$ : hydraulic diffusivity
- $\kappa_t$ : thermal diffusivity

In [18]:
#### set all material to basement material initially
index = df_layers['mat index'][df_layers['Base Surface'].str.contains('basement', case=False)].iloc[0]

### assign material variables
materialIndex.data[:] = index

### add in layer properties
fn_hydraulicDiffusivity.data[:] = (df_layers['ogia conductivity max (m/day)'].iloc[index] * u.meter/u.day).to(u.meter/u.second).magnitude

hydraulicDiffusivity.data[:] = (df_layers['ogia conductivity max (m/day)'].iloc[index] * u.meter/u.day).to(u.meter/u.second).magnitude

thermalDiffusivity.data[:] = df_layers['thermal conductivity'].iloc[index]

a_exponent.data[:] = df_layers['a (T)'].iloc[index]

heatProduction.data[:] = df_layers['Heat production (W/m3)'].iloc[index]

porosity.data[:] = df_layers['Porosity (%Vol)'].iloc[index] / 100 # from % to a value between 0 and 1


In [19]:
### gf to smooth the surfaces
from scipy.ndimage import gaussian_filter

mask_layer = np.ones(swarm.data.shape[0], dtype=bool)



## starting from the surface and going deeper with each layer
## skip topography as mesh is already deformed to topo
for index in df_layers.index[:]:
    row = df_layers.loc[index]
    
    # load surface
    with np.load(surface_filename.format(row['Layer name'])) as npz:
        layer_interp = RegularGridInterpolator((npz['y'], npz['x']), gaussian_filter(np.flipud(npz['data']), sigma=1))
        
    # interpolate surface to cell centroids
    z_interp = layer_interp((cellCentroid.data[:,1], cellCentroid.data[:,0]))
    z_interp_mesh = layer_interp((mesh.data[:,1], mesh.data[:,0]))
    
    # assign index to swarm particles which are below the current surface
    # if they are above the surface then we are done.
#     mask_layer[cellCentroid.data[:,2] > z_interp] = False
    mask_layer = cellCentroid.data[:,2] < z_interp
    mask_layer_mesh = mesh.data[:,2] < z_interp_mesh
    
    ### assign material variables
    materialIndex.data[mask_layer] = index
    
    ### add in layer properties
    fn_hydraulicDiffusivity.data[mask_layer] = (df_layers['ogia conductivity max (m/day)'].iloc[index] * u.meter/u.day).to(u.meter/u.second).magnitude

    hydraulicDiffusivity.data[mask_layer] = (df_layers['ogia conductivity max (m/day)'].iloc[index] * u.meter/u.day).to(u.meter/u.second).magnitude
    
    thermalDiffusivity.data[mask_layer] = df_layers['thermal conductivity'].iloc[index]
    
    a_exponent.data[mask_layer] = df_layers['a (T)'].iloc[index]
    
    heatProduction.data[mask_layer] = df_layers['Heat production (W/m3)'].iloc[index]
    
    porosity.data[mask_layer_mesh] = df_layers['Porosity (%Vol)'].iloc[index] / 100 # from % to a value between 0 and 1

    
    
    if uw.mpi.rank == 0:
        print("Layer {:2d}  {}".format(index, row['Name Aquifer/Aquitard']))
    
    
    

Layer  1  Shallow aquifer
Layer  2  Lake Eyre
Layer  3  Winton-Mackunda
Layer  4  Rolling Downs
Layer  5  Cadna-owie-Hooray
Layer  6  Westbourne
Layer  7  Adori-Springbok
Layer  8  Birkhead-Walloon
Layer  9  Hutton
Layer 10  Evergreen-Poolowanna
Layer 11  Precipice
Layer 12  Triassic Basins


In [20]:
# +
swarm_topography = topo_interp((cellCentroid.data[:,1],cellCentroid.data[:,0]))
mesh_topography  = local_topography

beta = 9.3e-3
depth = -1.0*(cellCentroid.data[:,2] - swarm_topography)
depth = np.clip(depth, 0.0, zmax-zmin)

depth_mesh = -1.0*(mesh.data[:,2] - mesh_topography)
depth_mesh = np.clip(depth_mesh, 0.0, zmax-zmin)

# +
Storage = 1.
rho_water = 1000.
c_water = 4e3
coeff = rho_water*c_water

# if deformedmesh:
#     g = uw.function.misc.constant((0.,0.,-1.))
# else:
#     g = uw.function.misc.constant((0.,0.,0.))

g = uw.function.misc.constant((0.,0.,0.))

gwPressureGrad = gwHydraulicHead.fn_gradient

gMapFn = -g*rho_water*Storage

In [21]:
fn_thermalDiffusivity = thermalDiffusivity*(298.0/temperatureField)**a_exponent
fn_source = uw.function.math.dot(-1.0*coeff*velocityField, temperatureField.fn_gradient) + heatProductionField

In [22]:
# groundwater solver
gwadvDiff = uw.systems.SteadyStateDarcyFlow(
                                            velocityField    = velocityField, \
                                            pressureField    = gwHydraulicHead, \
                                            fn_diffusivity   = fn_hydraulicDiffusivity, \
                                            conditions       = [gwPressureBC], \
                                            fn_bodyforce     = g, \
                                            voronoi_swarm    = swarm, \
                                            swarmVarVelocity = swarmVelocity)


# heatflow solver
heateqn = uw.systems.SteadyStateHeat( temperatureField = temperatureField, \
                                      fn_diffusivity   = fn_thermalDiffusivity, \
                                      fn_heating       = heatProduction, \
                                      conditions       = temperatureBC \
                                      )

In [23]:
### solve the groudwater flow
gwsolver = uw.systems.Solver(gwadvDiff)


### solve the heat flow
heatsolver = uw.systems.Solver(heateqn)

### create a copy of the original temperature field before solving
temperatureField0.data[:] = temperatureField.data[:]


gwsolver.solve()
heatsolver.solve(nonLinearIterate=False)

In [24]:
pressureField = gwHydraulicHead.copy(deepcopy=True)
pressureField.data[:] -= zCoordFn.evaluate(mesh)

In [25]:
## calculate velocity from Darcy velocity
# velocityField.data[:] /= np.clip(fn_porosity(depth_mesh*1e-3, porosity, 0.071, 5.989), 0.0, 1.0).reshape(-1,1)
velocityField.data[:] /= porosity.data[:]#np.clip(fn_porosity(depth_mesh*1e-3, 0.474, 0.071, 5.989), 0.0, 1.0).reshape(-1,1)

In [26]:
# temperature-dependent conductivity
temperatureField.data[:] = np.clip(temperatureField.data, Tmin, Tmax)
temperatureField.data[topWall] = Tmin
temperatureField.data[bottomWall] = Tmax

In [27]:
# dummy mesh variable
phiField        = mesh.add_variable( nodeDofCount=1 )
heatflowField   = mesh.add_variable( nodeDofCount=3 )


# calculate heat flux
thermalDiffusivity.data[:] = fn_thermalDiffusivity.evaluate(swarm)
kTproj = uw.utils.MeshVariable_Projection(phiField, thermalDiffusivity, swarm)
kTproj.solve()

heatflowField.data[:] = temperatureField.fn_gradient.evaluate(mesh) * -phiField.data.reshape(-1,1)

## Save to HDF5

In [28]:
xdmf_info_mesh  = mesh.save(simulation_directory+'mesh.h5')
xdmf_info_swarm = swarm.save(simulation_directory+'swarm.h5')

xdmf_info_matIndex = materialIndex.save(simulation_directory+'materialIndex.h5')
materialIndex.xdmf(simulation_directory+'materialIndex.xdmf', xdmf_info_matIndex,
                   'materialIndex', xdmf_info_swarm, 'TheSwarm')


# dummy mesh variable
phiField = mesh.add_variable( nodeDofCount=1 )


for xdmf_info,save_name,save_object in [(xdmf_info_swarm, 'materialIndexSwarm', materialIndex),
                                        (xdmf_info_swarm, 'hydraulicDiffusivitySwarm', fn_hydraulicDiffusivity),
                                        (xdmf_info_swarm, 'thermalDiffusivitySwarm', thermalDiffusivity),
                                        (xdmf_info_swarm, 'heatProductionSwarm', heatProduction),
                                        (xdmf_info_mesh, 'temperatureField', temperatureField),
                                        (xdmf_info_mesh, 'velocityField', velocityField),
                                        (xdmf_info_mesh, 'hydraulicHeadField', gwHydraulicHead),
                                        ]:
    
    xdmf_info_var = save_object.save(simulation_directory+save_name+'.h5')
    save_object.xdmf(simulation_directory+save_name+'.xdmf', xdmf_info_var, save_name, xdmf_info, 'TheMesh')

    if save_name.endswith("Swarm"):
        # project swarm variables to the mesh
        hydproj = uw.utils.MeshVariable_Projection(phiField, save_object, swarm)
        hydproj.solve()

        field_name = save_name[:-5]+'Field'
        xdmf_info_var = phiField.save(simulation_directory+field_name+'.h5')
        phiField.xdmf(simulation_directory+field_name+'.xdmf',
                      xdmf_info_var, field_name,
                      xdmf_info_mesh, "TheMesh")

In [127]:
### Doesn't work in parallel but can be done in post processing

# ### create a dataframe to caculate stats easily
# vel_df = pd.DataFrame()

# ### put velocity data into dataframe 
# vel_df['vel x'] = velocityField.data[:,0]
# vel_df['vel y'] = velocityField.data[:,1]
# vel_df['vel z'] = velocityField.data[:,2]
# vel_df.insert(0, 'mat', materialIndex.evaluate(mesh))


# ### group by material and then caculate stats for each material
# vel_stats = pd.DataFrame()
# vel_stats = vel_df.groupby("mat").describe()

# ### maps material index to layer names
# layerNameDic = dict(zip(df_layers['mat index'], df_layers['Name Aquifer/Aquitard']))
# layerDescriptionDic = dict(zip(df_layers['mat index'], df_layers[df_layers.columns[4]]))

# ### adds column of layer names into the stats dataframe
# vel_stats.insert(0, 'Name Aquifer/Aquitard', vel_stats.index.map(layerNameDic))
# vel_stats.insert(1, 'Hydrostratigraphy (2012 Atlas)', vel_stats.index.map(layerDescriptionDic))

# ### save values
# vel_stats.to_csv(simulation_directory+'velocity_stats.csv')